In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing as pre, tree
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import pickle

In [2]:
data = pd.read_csv('data.csv')
data.drop(['duration','day','month'],axis = 1, inplace = True)

In [3]:
for col in ['default','housing','loan','deposit']: # convert booleans to values
    data[col] = data[col].map({'yes':1,'no':0})

In [4]:
data = pd.get_dummies(data) # on-hot encoding for categorical data

In [5]:
test = data.sample(frac = 0.2, random_state = 42) # take 20% of the data and put it into a test dataset for validation
train = data.drop(test.index) # then drop from the original - we won't touch this until we're testing our model
train.to_csv('train.csv')
test = test.drop('deposit', axis = 1)
test.to_csv('test.csv')

In [6]:
trainset = pd.read_csv('train.csv', index_col = 0)

In [7]:
traindata = trainset.drop('deposit', axis = 1) # separate into train and test
test = trainset.filter(['deposit'], axis = 1)

In [8]:
scaler = pre.StandardScaler().fit(traindata)
pickle.dump(scaler,open('scaler.pkl', mode = 'wb'))
train = pd.DataFrame(scaler.transform(traindata), index = traindata.index, columns = traindata.columns)

C:\Users\Josep\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Josep\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
clf = tree.DecisionTreeClassifier(max_depth = 3, random_state = 42) # sets the parameter for the model

In [10]:
clf.fit(train, test) # trains the decision tree

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [11]:
pickle.dump(clf,open('model.pkl',mode = "wb"))
clf = pickle.load(open('model.pkl',mode = "rb"))

In [12]:
test = pd.read_csv('test.csv', index_col = 0)
scaled = pd.DataFrame(scaler.transform(test), index = test.index, columns = test.columns)

C:\Users\Josep\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  


In [13]:
test['deposit'] = clf.predict(scaled)

In [14]:
test.head()

,age,default,balance,housing,loan,campaign,pdays,previous,job_admin.,job_blue-collar,...,education_tertiary,education_unknown,contact_cellular,contact_telephone,contact_unknown,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,deposit
5527,65,0,744,0,0,1,-1,0,0,0,...,0,0,1,0,0,0,0,0,1,1
4541,38,0,1627,0,0,9,-1,0,0,1,...,0,0,0,0,1,0,0,0,1,0
1964,35,0,4963,1,0,1,300,1,0,1,...,0,0,1,0,0,1,0,0,0,0
5007,47,0,8229,0,0,2,-1,0,0,1,...,0,0,1,0,0,0,0,0,1,1
8928,35,0,1473,1,0,3,-1,0,0,0,...,1,0,0,0,1,0,0,0,1,0
